In [1]:
#importing pandas library and readinginsurance data set
import pandas as pd
df =pd.read_csv('insurance_pre.csv')

In [2]:
df
#from this data set age , sex,ami , children,smoker and all input charges is output

,age,sex,bmi,children,smoker,charges
0,19,female,27.900,0,yes,16884.92400
1,18,male,33.770,1,no,1725.55230
2,28,male,33.000,3,no,4449.46200
3,33,male,22.705,0,no,21984.47061
4,32,male,28.880,0,no,3866.85520
...,...,...,...,...,...,...
1333,50,male,30.970,3,no,10600.54830
1334,18,female,31.920,0,no,2205.98080
1335,18,female,36.850,0,no,1629.83350
1336,21,female,25.800,0,no,2007.94500


In [3]:
#smoker and sex is categorical column so using get_dummies function performaing one hot encoding to change it to number
df = pd.get_dummies(df,drop_first=True)
df

,age,bmi,children,charges,sex_male,smoker_yes
0,19,27.900,0,16884.92400,0,1
1,18,33.770,1,1725.55230,1,0
2,28,33.000,3,4449.46200,1,0
3,33,22.705,0,21984.47061,1,0
4,32,28.880,0,3866.85520,1,0
...,...,...,...,...,...,...
1333,50,30.970,3,10600.54830,1,0
1334,18,31.920,0,2205.98080,0,0
1335,18,36.850,0,1629.83350,0,0
1336,21,25.800,0,2007.94500,0,0


In [4]:
df.columns

Index(['age', 'bmi', 'children', 'charges', 'sex_male', 'smoker_yes'], dtype='object')

In [5]:
# splitting input and output column
x_independent = df[['age', 'bmi', 'children',  'sex_male', 'smoker_yes']]
y_depenedent = df[['charges']]

In [6]:
#splitting train and test set
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x_independent,y_depenedent,test_size=0.30,random_state=0)

In [7]:
#converting into standard form
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x_train_s = sc.fit_transform(x_train) 
x_test_s = sc.transform(x_test)

In [12]:
pip install xgboost

  Obtaining dependency information for xgboost from https://files.pythonhosted.org/packages/32/10/4689bda37403f7dd029d550c4446e0097c2f33b8ae877b235e76d5c49bc2/xgboost-2.0.0-py3-none-win_amd64.whl.metadata
   ---------------------------------------- 0.0/99.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/99.7 MB 1.3 MB/s eta 0:01:18
   ---------------------------------------- 0.0/99.7 MB 1.3 MB/s eta 0:01:18
   ---------------------------------------- 0.0/99.7 MB 1.3 MB/s eta 0:01:18
   ---------------------------------------- 0.1/99.7 MB 297.7 kB/s eta 0:05:35
   ---------------------------------------- 0.1/99.7 MB 403.5 kB/s eta 0:04:07
   ---------------------------------------- 0.1/99.7 MB 403.5 kB/s eta 0:04:07
   ---------------------------------------- 0.1/99.7 MB 403.5 kB/s eta 0:04:07
   ---------------------------------------- 0.1/99.7 MB 284.4 kB/s eta 0:05:51
   ---------------------------------------- 0.2/99.7 MB 378.3 kB/s eta 0:04:24
   -----------------

In [17]:
# create function
#xgboosting checking 
def XGBoost_checking(x_train,x_test,y_train,y_test,x_train_s,x_test_s):
    ''' input should be x_train , x_test,y_train y_test in standard form and acutal form
    this function will create XGboosting Regressor model using all possible hyper parameter in normal form and standard form
     and make a note of all combo r2 core
     return value is 2 data frame 1 st data frame will give all reading 
     2nd data frame will give which combo is working good that combo details     
      
    '''
    import xgboost as xg
    from sklearn.metrics import r2_score
    XGBoost_score_df=pd.DataFrame(columns=['std/normal','n_estimators','r_score'])
    n_estimators = [50,100,150,200,300]
    form=['Standard','Normal']
    for f in form:
        for i in n_estimators:
            XGBoost_reg=xg.XGBRegressor(n_estimators=i,seed=123)
            if(f=='Standard'):
                XGBoost_reg.fit(x_train_s,y_train)
                y_pred = XGBoost_reg.predict(x_test_s)
            else:
                XGBoost_reg.fit(x_train,y_train)
                y_pred = XGBoost_reg.predict(x_test)                
            XGBoost_r_score = r2_score(y_test,y_pred)
            XGBoost_score_df.loc[len(XGBoost_score_df.index)] = [f, i , XGBoost_r_score]
            max_score= XGBoost_score_df[XGBoost_score_df['r_score']==XGBoost_score_df['r_score'].max()]
    return (XGBoost_score_df,max_score)   
            

In [18]:
#checking Adaboosting model
XGBoost_all,XGBoost_max=XGBoost_checking(x_train,x_test,y_train,y_test,x_train_s,x_test_s)

In [20]:
XGBoost_max

,std/normal,n_estimators,r_score
0,Standard,50,0.835987
5,Normal,50,0.835987


In [23]:
XGBoost_all

,std/normal,n_estimators,r_score
0,Standard,50,0.835987
1,Standard,100,0.821334
2,Standard,150,0.814161
3,Standard,200,0.810942
4,Standard,300,0.808799
5,Normal,50,0.835987
6,Normal,100,0.821334
7,Normal,150,0.814161
8,Normal,200,0.810942
9,Normal,300,0.808799


In [24]:
# this will train xgboosting model with given x and y train field with given n estimators
def XGBoost_Model(x_train,y_train,n_estimators):
    import xgboost as xg
    XGBoost_reg=xg.XGBRegressor(n_estimators=n_estimators,seed=123)          
    XGBoost_reg.fit(x_train_s,y_train)                         
    return(XGBoost_reg)
            

In [25]:
XGBoost_reg=XGBoost_Model(x_train,y_train,50)

In [27]:
result = XGBoost_reg.predict([[19,27.900,0,0,1]])

In [28]:
result

array([18509.57], dtype=float32)